In [131]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install requests bs4 datetime pandas numpy

In [126]:
import requests  
from bs4 import BeautifulSoup
from datetime import date
import pandas as pd

def stakeholder(symbol):
    page = requests.get("https://www.set.or.th/set/factsheet.do?symbol="+ symbol +"&ssoPageId=3&language=th&country=TH")
    soup = BeautifulSoup(page.content, 'html.parser')
    soup = soup.table.find_all('table')[2].find_all('table')
    
    price = 0
    stakeholder = None
    for table in soup:
        strong = table.tr.strong
        if strong is None:
            continue
            
        if strong.text.find('Price (บาท)') >= 0:
            price = table.findAll('tr')[1].td.text
            continue

        if strong.text.find('ผู้ถือหุ้นรายใหญ่') == -1: 
            continue

        stakeholder = table
        pass
    
    data = []
    for tr in stakeholder.findAll('tr')[1:]:
        td = tr.findAll('td')
        data.append([
            date.today(), symbol, td[1].text.strip(), td[2].text, price, td[3].text
        ])
        
    return data

pd.DataFrame(stakeholder('TMB'))

,0,1,2,3,4,5
0,2019-10-14,TMB,กระทรวงการคลัง,"11,364,282,005",1.49,25.92
1,2019-10-14,TMB,ING BANK N.V.,"10,970,893,359",1.49,25.02
2,2019-10-14,TMB,บริษัท ไทยเอ็นวีดีอาร์ จำกัด,"4,802,062,226",1.49,10.95
3,2019-10-14,TMB,SOUTH EAST ASIA UK (TYPE C) NOMINEES LIMITED,"757,735,144",1.49,1.73
4,2019-10-14,TMB,นาง สมพร จึงรุ่งเรืองกิจ,"633,000,000",1.49,1.44
5,2019-10-14,TMB,กองทัพบก,"546,487,860",1.49,1.25
6,2019-10-14,TMB,MORGAN STANLEY & CO. INTERNATIONAL PLC,"406,144,764",1.49,0.93
7,2019-10-14,TMB,STATE STREET EUROPE LIMITED,"364,694,576",1.49,0.83
8,2019-10-14,TMB,N.C.B.TRUST LIMITED-NORGES BANK 5,"277,659,000",1.49,0.63
9,2019-10-14,TMB,GIC PRIVATE LIMITED,"229,810,639",1.49,0.52


In [130]:
!pwd
soup = BeautifulSoup(open('./set50.html'), 'html.parser')
setlist = [];
for tr in soup.findAll('tr')[1:]:
    td = tr.findAll('td')
    setlist.append(td[0].text.strip())

symbols = setlist;

stakeholder_data = []
for symbol in symbols:
    stakeholder_data = stakeholder_data + stakeholder(symbol)

pd.DataFrame(stakeholder_data, columns=["datetime", 'symbol', 'name',  "qty", "price", '%']).to_csv("stakeholder.csv")

/Users/user/work/jupytor


FileNotFoundError: [Errno 2] No such file or directory: './set50.html'

In [124]:
accounts = {}
for sh in stakeholder_data:
    symbol = sh[1]
    account = sh[2]
    qty = sh[3].replace(',', '')
    price = sh[4].replace(',', '')
    value = float(price) * float(qty)
    if account in accounts:
        target = accounts[account]
        accounts[account] = [
            target[0] + value,
            target[1] + 1
        ]
        continue
    accounts[account] = [value, 1]
    
accounts_arr = []
for account in accounts.keys():
    accounts_arr.append([
        account,
        '{:20,.2f}'.format(accounts[account][0]),
        accounts[account][1]
    ])

import numpy as np

accounts_arr.sort(key=lambda elem : elem[1],reverse=True)
df = pd.DataFrame(accounts_arr[0:10], columns=["account", 'value', 'count'])
df

,account,value,count
0,กระทรวงการคลัง,"1,427,531,002,874.95",3
1,บริษัท ไทยเอ็นวีดีอาร์ จำกัด,"976,912,725,477.91",47
2,บริษัท ปตท. จำกัด (มหาชน),"561,078,437,911.25",5
3,SOUTH EAST ASIA UK (TYPE C) NOMINEES LIMITED,"304,876,049,745.13",40
4,บริษัท อินทัช โฮลดิ้งส์ จำกัด (มหาชน),"275,421,048,000.00",1
5,STATE STREET EUROPE LIMITED,"242,784,102,551.51",35
6,พระบาทสมเด็จพระวชิรเกล้าเจ้าอยู่หัว,"241,447,717,765.00",2
7,บริษัท ซี.พี.เมอร์แชนไดซิ่ง จำกัด,"220,634,604,000.00",1
8,สำนักงานประกันสังคม,"199,854,677,153.45",32
9,SINGTEL STRATEGIC INVESTMENTS PTE LTD.,"158,779,211,000.00",1
